In [ ]:
 [code] {"execution":{"iopub.status.busy":"2025-04-18T18:57:02.757048Z","iopub.execute_input":"2025-04-18T18:57:02.757205Z","iopub.status.idle":"2025-04-18T18:57:33.966101Z","shell.execute_reply.started":"2025-04-18T18:57:02.757190Z","shell.execute_reply":"2025-04-18T18:57:33.965307Z"}}

In [ ]:
!pip install caer canaro

In [ ]:
!pip install caer

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import canaro

In [ ]:
import os 
import caer 
import numpy as np
import cv2 as cv 
import gc

In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r'/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
#sort the 
char_dict  = caer.sort_dict(char_dict,  descending = True)
char_dict

In [ ]:
characters = []
count  = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 5:
        break
characters

In [ ]:





train  = caer.preprocess_from_dir(char_path , characters , channels = channels , IMG_SIZE = IMG_SIZE)

In [ ]:
len(train)

In [ ]:
featureSet,labels =caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical


#normalize the featureSet  ==>(0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels , len(characters))

In [ ]:
x_train , x_val ,y_train, y_val = caer.train_val_split(featureSet , labels, val_ratio = 0.2)

In [ ]:
BATCH_SIZE  =32
EPOCHS = 5


datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train,batch_size = BATCH_SIZE)

In [ ]:
featureSet,labels =caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical


#normalize the featureSet  ==>(0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels , len(characters))

In [ ]:
x_train , x_val ,y_train, y_val = caer.train_val_split(featureSet , labels, val_ratio = 0.2)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
model = models.Sequential()

In [ ]:
from keras import layers, models

model = models.Sequential()

# Use Input layer to specify the input shape instead
model.add(layers.InputLayer(input_shape=(80, 80, 1)))  # Specify input shape here

# Then add your Conv2D layers
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output to feed into dense layers
model.add(layers.Flatten())

In [ ]:
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='sigmoid'))

In [ ]:
learning_rate = 0.001
momentum = 0.9
nesterov = True
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(train_gen,steps_per_epoch = len(x_train),
                    epochs =EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=len(y_val),
                    callbacks = callbacks_list)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [54]:
test_path = '/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_14.jpg'
img = cv.imread(test_path)  # or .png
def prepare(img):
    img  = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img= cv.resize(img,IMG_SIZE)
    img = caer.reshape(img,IMG_SIZE,1)
    return img

In [55]:
predictions = model.predict(prepare(img))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [ ]:
predictions

In [56]:
print(characters[np.argmax(predictions[0])])

lisa_simpson
